In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
baseModel = VGG16(weights="imagenet", include_top=False,
input_tensor=Input(shape=(25,50, 3)))



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'block5_pool/MaxPool' (op: 'MaxPool') with input shapes: [?,1,3,512].

In [ ]:
# construct the head of the model that will be placed on top of the the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
